In [1]:
from missing_bids import auction_data, analytics, environments
hist_plot = auction_data.hist_plot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

sns.set_style('white')
%matplotlib inline

In [2]:
# getting data directory

path_data = '/home/sylvain/Dropbox/Econ/papiers/gameTheory/missing_bids/data/data_for_missing_bids_figures'
#None #path/to/data (if you know it, otherwise, we'll find it below)

if path_data is None:
    name = 'data_for_missing_bids_figures'
    for root, dirs, _ in os.walk('/'):
        if name in dirs:
            path_data = os.path.join(root, name)
            break
        
print('data located at \n\t{}'.format(path_data))
path_figures = os.path.join(path_data, 'figures')

if not os.path.exists(path_figures):
    os.makedirs(path_figures)

data located at 
	/home/sylvain/Dropbox/Econ/papiers/gameTheory/missing_bids/data/data_for_missing_bids_figures


In [3]:
# set global optimization parameters

num_points = 3000
number_iterations = 5
confidence_level = .95
list_ks = [0.5, 1, 1.5, 2]

In [4]:
def compute_minimization_solution(data, deviations, max_markup=.5):
    demands = [data.get_counterfactual_demand(rho) for rho in deviations]
    solutions = []
    filter_ties = auction_data.FilterTies(tolerance=.0001)
    filtered_data = filter_ties(data)
    share_ties = filter_ties.get_ties(data).mean()

    for k in list_ks:
        constraints = [environments.MarkupConstraint(max_markup=max_markup, min_markup=.02), 
                   environments.InformationConstraint(k=k, sample_demands=demands)]

        min_collusion_solver = analytics.MinCollusionIterativeSolver(
            data=filtered_data, 
            deviations=deviations, 
            metric=analytics.IsNonCompetitive, 
            plausibility_constraints=constraints, 
            num_points=num_points, 
            seed=0, 
            project=False, 
            filter_ties=None,
            number_iterations=number_iterations,
            confidence_level=confidence_level,
            moment_matrix=auction_data.moment_matrix(deviations, 'slope'),
            moment_weights=np.array([0, 0, 1])
        )

        share_collusive = min_collusion_solver.result.solution    
        solutions.append(1 - share_collusive)
        del min_collusion_solver
    del filtered_data
    return np.array(solutions), share_ties

In [5]:
def pretty_plot(title, list_solutions, labels, mark=['k.:', 'k.-']):
    plt.title(title)
    for i, (solutions, label) in enumerate(zip(list_solutions, labels)):
        plt.plot(list_ks, solutions, mark[i], label=label)
    plt.legend(loc='lower right')
    plt.axis([list_ks[0], list_ks[-1], 0, 1.05])
    plt.xlabel('k')
    plt.ylabel('share of competitive histories')
    plt.savefig(os.path.join(path_figures, '{}.pdf'.format(title)))
    plt.show()

In [ ]:
national_data = auction_data.AuctionData('../tests/reference_data/sample_with_firm_rank.csv')
filter_ties = auction_data.FilterTies(tolerance=.0001)
filtered_data = filter_ties(national_data)

comp_histories_firms_2 = []

#firms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
deviations = [-.025, .0, .001]
count = 1

while count < 11:
    filtered_data_firm = auction_data.AuctionData.from_clean_bids(
    filtered_data.df_bids.loc[filtered_data.data.rank2 == count])
    demand_firm = [filtered_data_firm.get_counterfactual_demand(rho) for rho in deviations]
    print(demand_firm)
    constraints = [environments.MarkupConstraint(max_markup=.5, min_markup=.02), 
    environments.InformationConstraint(k=1, sample_demands=demand_firm)]

    min_collusion_solver = analytics.MinCollusionIterativeSolver(
        data=filtered_data_firm, 
        deviations=deviations, 
        metric=analytics.IsNonCompetitive, 
        plausibility_constraints=constraints, 
        num_points=30000.0, 
        seed=0, 
        project=False, 
        filter_ties=None,
        number_iterations=500,
        confidence_level=.95,
        moment_matrix=auction_data.moment_matrix(deviations, 'slope'),
        moment_weights=np.array([0, 0, 1])
    )

    share = min_collusion_solver.result.solution
    print(1 - share) 
    comp_histories_firms_2.append(1 - share)
    count = count + 1
    
print(comp_histories_firms_2)

[0.2633498306850742, 0.17712946079708258, 0.17712946079708258]
0.5983225146928144
[0.15198956294846705, 0.1004566210045662, 0.1004566210045662]


In [31]:
print(comp_histories_firms_2)

[0.5983225785780911, 0.6802485938054595, 0.5983227188764317, 0.999999999996455, 0.6802486191487804, 0.12868268014296702, 0.1540303084010337, 0.1484759388140957, 0.6802488600281172, 0.15403031065044115, 0.5983224688964097, 0.6802491370470429, 0.14847594652930796, 0.1484759388998611, 0.9457864553052578, 0.1540303097504846, 0.6802487829774598, 0.14847594204754122, 0.1484759389661695, 1.0000000000062657, 0.16394525190076736, 1.000000000007576, 0.16394525271129934, 1.000000000000639, 0.1639452517060359, 0.9627616318719026, 0.16394526207861293, 0.5983224878333374, 0.16394525409827598, 0.598322458065558]
